In [1]:
import math
import os
import re
from os import path

import spacy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

DIRETORIO_PDF = 'BauruPDF/'
DIRETORIO_TXT = 'BauruTXT/'
DIRETORIO_DOC = 'BauruDOC/'
DIRETORIO_PAR = 'BauruPAR/'
DIRETORIO_JSON = 'BauruJSON/'

nlp_exclude_list = ['parser',
                    'ner',
                    'entity_linker',
                    'entity_ruler',
                    'textcat',
                    'textcat_multilabel',
                    'lemmatizer',
                    'trainable_lemmatizer',
                    'morphologizer',
                    'attribute_ruler',
                    'senter',
                    'sentencizer',
                    'tok2vec',
                    'transformer']
nlp = spacy.load('pt_core_news_lg', exclude = nlp_exclude_list)

In [2]:
processos = []
editais = []
for filename in tqdm(os.listdir(DIRETORIO_JSON)):
    with open(path.join(DIRETORIO_JSON, filename), 'r') as file:
        licitacao_json = json.load(file)
    
    processos_licitacao = []
    
    
    if 'processo' in licitacao_json:
        processo_num = int(licitacao_json['processo']['numero'].replace('.', ''))
        processo_ano = int(licitacao_json['processo']['ano'])
        
        processos_licitacao.append((processo_num, processo_ano))

    if 'processos_apensados' in licitacao_json:
        for processo in licitacao_json['processos_apensados']:
            processo_num = int(processo['numero'].replace('.', ''))
            processo_ano = int(processo['ano'])

            processos_licitacao.append((processo_num, processo_ano))

    if 'processo' not in licitacao_json and 'processos_apensados' not in licitacao_json:
        processos_licitacao.append((-1, -1))
    
    for processo_num, processo_ano in processos_licitacao:
        processos.append({
            'identificador': licitacao_json['identificador'],
            'numero': processo_num,
            'ano': processo_ano,
        })
    
    editais_licitacao = []
    if 'documentos' in licitacao_json:
        encontrou_edital = False
        for documento in licitacao_json['documentos']:
            if documento['nome'] == 'Edital':
                encontrou_edital = True
                edital_num = int(documento['numero'])
                edital_ano = int(documento['ano'])
                editais_licitacao.append((edital_num, edital_ano))
        
        if not encontrou_edital:
            editais_licitacao.append((-1, -1))
        
    for edital_num, edital_ano in editais_licitacao:
        editais.append({
            'identificador': licitacao_json['identificador'],
            'numero': edital_num,
            'ano': edital_ano
        })


processos_licitacoes_df = pd.DataFrame(processos)
editais_licitacoes_df = pd.DataFrame(editais)
# print(processos_licitacoes_df)
# print(editais_licitacoes_df)

100%|██████████| 3522/3522 [00:24<00:00, 143.33it/s]


In [10]:
licitacoes_sem_edital = editais_licitacoes_df[(editais_df['numero'] == -1) & (editais_licitacoes_df['ano'] == -1)]['identificador']
processos_licitacoes_df.merge(licitacoes_sem_edital, on = 'identificador')

,identificador,numero,ano
0,6752,51646,2021
1,4343,43498,2017
2,7349,193912,2021
3,7349,193921,2021
4,5483,65258,2019
...,...,...,...
202,5010,57826,2018
203,7204,16500,2022
204,3841,28569,2016
205,4764,26167,2018


In [3]:
re_processo = re.compile('processo', flags = re.IGNORECASE)
re_edital = re.compile('edital', flags = re.IGNORECASE)
extrator_numero_ano = re.compile('((?P<numero>\d+(\.\d{3})?)/(?P<ano>\d{2,4}))')

look_ahead = 20

processos_encontrados = []
editais_encontrados = []
for filename in tqdm(os.listdir(DIRETORIO_PAR)):
    with open(path.join(DIRETORIO_PAR, filename), 'r') as file:
        paragrafos = json.load(file)
    
    for p_num, paragrafo in enumerate(paragrafos):
        processo_matches = re_processo.finditer(paragrafo)
        edital_matches = re_edital.finditer(paragrafo)
        
        for processo_match in processo_matches:
            numero_match = extrator_numero_ano.search(paragrafo[processo_match.end():processo_match.end() + look_ahead])

            if numero_match != None:
                num = int(numero_match.group('numero').replace('.', ''))
                ano = int(numero_match.group('ano'))
                if ano < 100:
                    ano += 2000
                processos_encontrados.append({
                    'do': filename,
                    'paragrafo': p_num,
                    'numero': num,
                    'ano': ano
                })
        
        for edital_match in edital_matches:
            numero_match = extrator_numero_ano.search(paragrafo[edital_match.end():edital_match.end() + look_ahead])
            
            if numero_match != None:
                num = int(numero_match.group('numero').replace('.', ''))
                ano = int(numero_match.group('ano'))
                if ano < 100:
                    ano += 2000
                editais_encontrados.append({
                    'do': filename,
                    'paragrafo': p_num,
                    'numero': num,
                    'ano': ano
                })

processos_encontrados_df = pd.DataFrame(processos_encontrados)
editais_encontrados_df = pd.DataFrame(editais_encontrados)
# print(processos_encontrados_df)
# print(editais_encontrados_df)

100%|██████████| 941/941 [00:28<00:00, 32.67it/s]


In [4]:
# Processos/editais encontrados nos diários que aparecem nas licitações
processos_inner = processos_licitacoes_df.merge(processos_encontrados_df, on = ['numero', 'ano'], how = 'inner')
editais_inner = editais_licitacoes_df.merge(editais_encontrados_df, on = ['numero', 'ano'], how = 'inner')
# print(processos_inner)
# print(editais_inner)

In [11]:
processos_licitacoes_df.merge(editais_licitacoes_df, how = 'inner', on = 'identificador', suffixes = ('_processo', '_edital'))

,identificador,numero_processo,ano_processo,numero_edital,ano_edital
0,7169,189247,2021,36,2022
1,4178,56944,2016,126,2017
2,4686,7050,2018,86,2018
3,4958,48991,2018,340,2018
4,6829,74409,2021,342,2021
...,...,...,...,...,...
3639,3718,9205,2016,273,2016
3640,5963,158506,2019,85,2020
3641,5582,14092,2019,295,2019
3642,6494,70392,2020,54,2021


In [22]:
licitacoes_em_dos = processos_inner.merge(editais_inner, how = 'inner', on = ['identificador', 'do', 'paragrafo'], suffixes = ('_processo', '_edital'))
licitacoes_em_dos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10786 entries, 0 to 10785
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   identificador    10786 non-null  object
 1   numero_processo  10786 non-null  int64 
 2   ano_processo     10786 non-null  int64 
 3   do               10786 non-null  object
 4   paragrafo        10786 non-null  int64 
 5   numero_edital    10786 non-null  int64 
 6   ano_edital       10786 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 674.1+ KB


In [23]:
candidatos_em_dos = processos_encontrados_df.merge(editais_encontrados_df, how = 'inner', on = ['do', 'paragrafo'], suffixes = ('_processo', '_edital'))
candidatos_em_dos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13326 entries, 0 to 13325
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   do               13326 non-null  object
 1   paragrafo        13326 non-null  int64 
 2   numero_processo  13326 non-null  int64 
 3   ano_processo     13326 non-null  int64 
 4   numero_edital    13326 non-null  int64 
 5   ano_edital       13326 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 728.8+ KB


In [57]:
matches_df = candidatos_em_dos.merge(licitacoes_em_dos,
                                     how = 'inner',
                                     on = ['do', 'paragrafo', 'numero_processo', 'ano_processo', 'numero_edital', 'ano_edital'],
                                     suffixes = ('_candidato', '_licitacao'))
matches_df_temp = matches_df[(matches_df['numero_edital'] == 351) & (matches_df['ano_edital'] == 2018)]
print(matches_df_temp)
for _, item in matches_df_temp.iterrows():
    with open(path.join(DIRETORIO_PAR, item['do']), 'r') as file:
        paragrafos = json.load(file)
    print(item['do'], item['paragrafo'])
    print(paragrafos[item['paragrafo']])
    print()


                                    do  paragrafo  numero_processo  \
4009   do_20180925_3024.pdf.xhtml.json        575            45069   
5175   do_20181124_3046.pdf.xhtml.json        590            45069   
8109   do_20180823_3011.pdf.xhtml.json        377            45069   
8124   do_20181004_3028.pdf.xhtml.json       1994            45069   
10630  do_20181018_3033.pdf.xhtml.json       1641            45069   
10808  do_20180920_3022.pdf.xhtml.json        692            45069   
12756  do_20180913_3019.pdf.xhtml.json        875            45069   

       ano_processo  numero_edital  ano_edital identificador  
4009           2017            351        2018          4969  
5175           2017            351        2018          4969  
8109           2017            351        2018          4969  
8124           2017            351        2018          4969  
10630          2017            351        2018          4969  
10808          2017            351        2018          4969 

In [4]:
def similaridade_cosseno(a, b):
    """
    Calcula a similaridade de cosseno entre dois dicts.
    Supõe-se que as chaves são iguais, compondo "vetores" a e b em um mesmo espaço.
    Como se trata de sacos de palavras, os vetores terão apenas valores inteiros positivos,
    logo a similaridade não poderão assumir similaridades negativas.
    
    :param a dict: "vetor" a
    :param b dict: "vetor" b
    
    :returns: valor no intervalo [0, 1] similaridade cosseno entre os "vetores" a e b.
    """
    
    assert type(a) == dict
    assert type(b) == dict
    assert a.keys() == b.keys()
    
    produto_escalar  = 0
    soma_a = 0
    soma_b = 0
    
    for k in a:
        produto_escalar  += a[k] * b[k]
        soma_a           += math.pow(a[k], 2)
        soma_b           += math.pow(b[k], 2)

    magnitude = math.sqrt(soma_a) * math.sqrt(soma_b)
    
    if abs(magnitude) < 1e-3:
        similaridade = 0
    else:
        similaridade = produto_escalar / magnitude
    
    return similaridade

def atualiza_chaves(d, ks):
    """
    Atualiza as chaves do dict d para conter também as chaves ks.
    
    :param d dict: dict a ser atualizado
    :param ks dict_keys: chaves a serem adicionadas em d
    
    :returns: dict d
    """
    
    for k in ks:
        if k not in d:
            d[k] = 0

    return d

def token_valido(token):
    """
    Verifica se um token é valido para ser incluído no saco de palavras
    
    :param token spacy.tokens.Token: token a ser verificado
    
    :returns: bool se é válido ou não
    """
    
    assert type(token) == spacy.tokens.Token
    
    return not (token.is_stop
               or token.is_space
               or token.is_punct
               or token.is_currency
               or token.is_quote
               or token.is_bracket
               or token.like_num
               or token.is_oov)

def monta_saco_palavras(doc, saco = None):
    """
    Monta/atualiza o saco de palavras de um doc com os tokens válidos da função token_valido.
    
    :param doc spacy.tokens.Doc: Doc a ser utilizado no saco de palavras.
    :keyword saco dict: saco de palavras a ser atualizado, ou criado, caso não especificado
    
    :returns: saco de palavras criado, ou atualizado
    """
    
    if saco == None:
        saco = dict()
    
    for token in doc:
        if token_valido(token):
            chave = token.norm_
            if chave not in saco:
                saco[chave] = 0
            saco[chave] += 1
        else:
            pass
    return saco

def calcula_similaridade_doc(doc, saco_alvo_template):
    """
    Calcula a similaridade do saco de um Doc com outro saco previamente gerado.
    
    :param doc spacy.tokens.Doc: Doc a ser analisado.
    :param saco_alvo_template dict: saco a ser comparado.
    
    :returns: similaridade dos sacos gerados
    """
    
    saco_alvo = saco_alvo_template.copy()
    saco_doc = {}
    
    atualiza_chaves(saco_doc, saco_alvo.keys())
    monta_saco_palavras(doc, saco_doc)
    atualiza_chaves(saco_alvo, saco_doc.keys())
    
    similaridade = similaridade_cosseno(saco_alvo, saco_doc)
    
    return similaridade

def calcula_similaridade_entre_docs(doc_a, doc_b):
    saco_a = monta_saco_palavras(doc_a)
    similaridade = calcula_similaridade_doc(doc_b, saco_a)
    
    return similaridade

In [24]:
samples = processos_licitacoes_df.sample(n = 10)
#print(processos_licitacoes_df.info())

for _, sample in samples.iterrows():
    identificador_licitacao = sample['identificador']
    numero_processo = sample['numero']
    ano_processo = sample['ano']
    print('##########################################')
    print(identificador_licitacao, numero_processo, ano_processo)
    
    with open(path.join(DIRETORIO_JSON, identificador_licitacao + '.json'), 'r') as file:
        licitacao = json.load(file)
    
    candidatos_df = processos_encontrados_df[(processos_encontrados_df['numero'] == numero_processo)
                                             & (processos_encontrados_df['ano'] == ano_processo)]
    
    if len(candidatos_df) == 0:
        print('Processo sem parágrafos candidatos')
        continue
    
    candidatos = []
    for indice, candidato in candidatos_df.iterrows():
        with open(path.join(DIRETORIO_PAR, candidato['do']), 'r') as file:
            paragrafos = json.load(file)
        candidatos.append({
            'do': candidato['do'],
            'paragrafo': candidato['paragrafo'],
            'texto_paragrafo': paragrafos[candidato['paragrafo']]
        })
    
    candidatos_df = pd.DataFrame(candidatos)
    
#     print(candidatos_df)
    
    if 'publicacoes' in licitacao:
        print(identificador_licitacao, len(licitacao['publicacoes']), 'publicações, ', len(candidatos_df), 'candidatos')

        similaridades = np.zeros((len(licitacao['publicacoes']), len(candidatos_df)))
        
        for num_publicacao, publicacao in enumerate(licitacao['publicacoes']):
#             print('Publicacao', num_publicacao)
#             print(publicacao['conteudo'])
            
            saco_publicacao = monta_saco_palavras(nlp(publicacao['conteudo']))
            
            for num_candidato, candidato in enumerate(candidatos_df['texto_paragrafo']):
                similaridade = calcula_similaridade_doc(nlp(candidato), saco_publicacao)
#                 print('Candidato', num_candidato, similaridade)
#                 print(candidato)
                
                similaridades[num_publicacao, num_candidato] = similaridade

        print(similaridades)
        
        for num, publicacao in enumerate(licitacao['publicacoes']):
            melhor_candidato = similaridades[num,:].argmax()
            similaridade = similaridades[num,:].max()
            
            print(melhor_candidato, similaridade)
            print(publicacao['conteudo'])
            print('-----------------------------------------------------')
            print(candidatos_df.iloc[melhor_candidato]['texto_paragrafo'])
            print('#####################################################')
    else:
        print(identificador_licitacao, 'Licitação sem Publicações')
    

##########################################
5008 46009 2018
5008 1 publicações,  11 candidatos
[[0.3073641  0.24945594 0.24945594 0.3073641  0.24945594 0.3073641
  0.05082556 0.24945594 0.3073641  0.68227742 0.82161835]]
10 0.8216183526543441
 NOTIFICAÇÃO DE HOMOLOGAÇÃO - 
Notificamos aos interessados no processo licitatório epigrafado que o julgamento e a classificação havida foram devidamente Homologados pelo Prefeito Municipal em 22/10/2018 às empresas FORTMIX COMERCIO DE CONCRETO LTDA E GUARANI INDUSTRIA COMERCIO E SERVIÇOS LTDA conforme abaixo: 
LOTE 1 – TUBOS DE CONCRETO ARMADO CLASSE PA-2 - COTA RESERVADA - somente Microempresa (ME), MEI (Micro Empreendedor)  e Empresa de Pequeno Porte (EPP) qualificadas como tais, nos termos do art. 3º da Lei Complementar nº 123/06 alterada pela Lei Complementar nº 147/14.
EMPRESA: FORTMIX COMÉRCIO DE CONCRETO LTDA     VALOR TOTAL DO LOTE 01: R$ 63.406,52
ITEM	UND	QTD	ESPECIFICAÇÕES MÍNIMAS	P. UNIT	MARCA
01	M	42	metros de tubo de concreto armado

4169 2 publicações,  70 candidatos
[[0.16352528 0.17169083 0.15420971 0.14926807 0.16258277 0.30640844
  0.31712087 0.38613847 0.29486938 0.47108452 0.48379129 0.378051
  0.30237106 0.77915565 0.70345718 0.30640844 0.31712087 0.38613847
  0.29486938 0.47658088 0.48379129 0.378051   0.30237106 0.16546056
  0.30974597 0.30501191 0.3750386  0.283048   0.41336759 0.47428725
  0.36675465 0.11233119 0.34662049 0.32173406 0.31712087 0.38613847
  0.29486938 0.42338126 0.48379129 0.378051   0.30237106 0.31532969
  0.11233119 0.18663764 0.18663764 0.35399539 0.22466238 0.16134912
  0.30974597 0.30501191 0.3750386  0.283048   0.41336759 0.47428725
  0.36675465 0.29013328 0.34662049 0.14721526 0.29936363 0.16645433
  0.16074561 0.15420971 0.15420971 0.16352528 0.86248232 0.15420971
  0.1564832  0.1777169  0.16958177 0.16485712]
 [0.18135318 0.18409566 0.17125693 0.165769   0.17337299 0.32465286
  0.32275327 0.39619331 0.30725196 0.51294425 0.50716873 0.39431785
  0.31425976 0.78516866 0.99159649 0

[[0.30108313 0.23954649 0.99772086 0.37530656 0.84022893 0.209916  ]
 [0.60955692 1.         0.24299338 0.49378032 0.2083177  0.36850841]]
2 0.9977208610650053
 NOTIFICAÇÃO DE HOMOLOGAÇÃO - ÓRGÃO: PREFEITURA MUNICIPAL DE BAURU - SECRETARIA MUNICIPAL DE SAÚDE
Processo: 106.597/2021 – Modalidade: Pregão Eletrônico SMS n° 281/2021 – Sistema de Registro de Preços – AMPLA PARTICIPAÇÃO – por meio da INTERNET – Tipo Menor Preço por item – Objeto: aquisição anual estimada de diversos medicamentos para o município. Aberto no dia 10/09/2021 às 14 h. Notificamos aos interessados no Processo licitatório epigrafado, que o julgamento e a classificação havidos foram devidamente homologados pela Prefeita Municipal em 22/09/2021, conforme segue abaixo:
ANTIBIOTICOS DO BRASIL LTDA
Item 02 - Cefalexina Monoidratada 50 Mg/ml (250 Mg/5 Ml), frasco 60 ml; à R$ 4,7921 unitário. Marca/Fabricante: Genérico/Antibióticos do Brasil
CENTERMEDI COMÉRCIO DE PRODUTOS HOSPITALARES LTDA
Item 01 - Budesonida 32mcg/dose;

In [55]:
#samples = processos_encontrados_df.merge(processos_licitacoes_df, how = 'inner', on = ['numero', 'ano']).sample(n = 10)
#samples = processos_licitacoes_df.sample(10)

re_edital = re.compile('edital', flags = re.IGNORECASE)
re_processo = re.compile('processo', flags = re.IGNORECASE)
re_modalidade = re.compile('modalidade', flags = re.IGNORECASE)
re_objeto = re.compile('objeto', flags = re.IGNORECASE)
re_interessado = re.compile('interessad(o|a)(s?)', flags = re.IGNORECASE)

extrator_numero_ano = re.compile('((?P<numero>\d+(\.\d{3})?)/(?P<ano>\d{2,4}))')

for _, sample in samples.iterrows():
#     with open(path.join(DIRETORIO_PAR, sample['do']), 'r') as file:
#         paragrafos = json.load(file)
    with open(path.join(DIRETORIO_JSON, sample['identificador'] + '.json'), 'r') as file:
        licitacao = json.load(file)
    
    if 'publicacoes' in licitacao:
        for publicacao in licitacao['publicacoes']:
            if publicacao['titulo'] == 'Abertura':
                paragrafo = publicacao['conteudo']
                break
            else:
                pass
    else:
        continue
                
#     paragrafo = paragrafos[sample['paragrafo']]
    
    m_edital = re_edital.search(paragrafo)
    m_processo = re_processo.search(paragrafo)
    m_modalidade = re_modalidade.search(paragrafo)
    m_objeto = re_objeto.search(paragrafo)
    m_interessado = re_interessado.search(paragrafo)
    
    #print(sample['do'], sample['paragrafo'], sample['numero'], sample['ano'])
    print(paragrafo)
    if m_edital != None:
        print('m_edital', m_edital.start(), m_edital.end())
    if m_processo != None:
        print('m_processo', m_processo.start(), m_processo.end())
    if m_modalidade != None:
        print('m_modalidade', m_modalidade.start(), m_modalidade.end())
    if m_objeto != None:
        print('m_objeto', m_objeto.start(), m_objeto.end())
    if m_interessado != None:
        print('m_interessado', m_interessado.start(), m_interessado.end())
    print()
    

 NOTIFICAÇÃO DE ABERTURA - ÓRGÃO: PREFEITURA MUNICIPAL DE BAURU - SECRETARIA MUNICIPAL DE SAÚDE
Processo: 15.292/2021 – Modalidade: Pregão Eletrônico SMS n° 212/2022 – EXCLUSIVA PARA PARTICIPAÇÃO DE ME’S EPP’S - por meio da INTERNET – Tipo Menor Preço por Lote – Objeto: aquisição de material – confecção de assentos almofadados e encostos almofadados em espuma e revestidos em material de courvin, visando à adequação e conforto de sofás em alvenaria da Divisão de Saúde Mental/CAPS Girassol em Bauru. A Data do Recebimento das Propostas será até dia 15/06/2022 às 9h - A abertura da Sessão dar-se-á no dia 15/06/2022 às 9h. – Pregoeira: Monica Alesandra de Oliveira. O Edital completo e informações poderão ser obtidos na Divisão de Compras e Licitações, Rua Gérson França, 7-49, 1º andar, Centro, CEP: 17015-200 – Bauru/SP, fone (14) 3104-1463/1465, ou pelo site www.bauru.sp.gov.br ou www.bec.sp.gov.br, OC 820900801002022OC00257, onde se realizará a sessão de pregão eletrônico, com os licitante

In [41]:
processos_licitacoes_df[(processos_licitacoes_df['numero'] == '2654')]
                        #& (processos_licitacoes_df['ano'] == '2018')]

,identificador,numero,ano
